# Mancala Game

In [35]:
#not used... yet
from IPython.display import clear_output
#clear_output()

In [36]:
import random

In [37]:
#objeto Tablero
    # tiene un tablero y mancalas
    # metodo para realizar jugada de P1 o P2
    # metodo para saber si fila x == 0
    # metodo para imprimir en pantalla el estado del juego

class Tablero:
    def __init__(self):
        #inicializa el tablero y mancala
        self.mancala = [0,0]
        self.board = [0]*2
        for i in range(2):
            self.board[i] = [4]*6
        self.over = False
    
    
    #devuelve True si la fila seleccionada (i [0,1]) está vacía
    def fila_Vacia(self,i):
        for j in range(6):
            if(self.board[i][j] != 0):
                return False
        return True
    
    def gameOver(self):
        if(self.fila_Vacia(0) or self.fila_Vacia(1)):
            self.over = True
    
    #realiza una jugada
    def makePlay(self, player, position):
        _position = position
        tokens = self.board[player][position] #obtengo todas las fichas en la casilla
        self.board[player][position] = 0 #remuevo todas las fichas de la casilla
        direction = player #0: backward ; 1: forward            
        while(tokens > 0):
            if(direction == 0):
                (tokens, _position) = self.run_backward(player,_position,tokens)
                direction = 1
            else:
                (tokens, _position) = self.run_forward(player,_position,tokens)
                direction = 0
        print("last position: ",_position)
        self.gameOver() ## verificar si se ha acabado el juego
        return direction, _position #### position tupla [fila, position]
                
                
            
    def run_backward(self, player, position, tokens):
        fill_c = position - 1 #la primera posicion a llenar
        _tokens = tokens
        #se toma en cuenta -1 por ser la casilla mancala para jugador 0
        while((_tokens > 0) and (fill_c >-2)):
            #not mancala
            if(fill_c >-1):
                self.board[0][fill_c] += 1
                _tokens -= 1
            #mancala
            else:
                if(player == 0):
                    self.mancala[0] += 1
                    _tokens -= 1
            fill_c -= 1 #next position
        return (_tokens, fill_c+1)
    
    def run_forward(self, player, position, tokens):
        fill_c = position + 1
        _tokens = tokens
        #se toma en cuenta 6 por ser la casilla mancala para jugador 1
        while((_tokens > 0) and (fill_c < 7)):
            #not mancala
            if(fill_c < 6):
                self.board[1][fill_c] += 1
                _tokens -= 1
            #mancala
            else:
                if(player == 1):
                    self.mancala[1] += 1
                    _tokens -= 1
            fill_c += 1 #next position
        return (_tokens,fill_c-1)
    
    def showBoard(self):
        #score
        print("_______________________ *** _______________________")
        print("Jugador 1: ", self.mancala[0],"\t\t\tJugador 2: ",self.mancala[1],"\n\n")
        #board
        for i in range(2):
            print("\t", end = '')
            for j in range(6):
                print(self.board[i][j], end = '')
                print("\t", end = '')
            print("\n")
                    
    

In [ ]:
#objeto IA
    # tiene metodo para realizar turno
    # tiene metodo para realizar monte carlo
    
    # por ahora tablero original es game
    # copia de game es tablero
    tablero0 = game
    for i in range(6):
        for j in range(n):
            ran = random.randint(0,5)
            tablero0.makePlay(i,ran)
            while(not game.over):
                u1 = random.randint(0,5)
                if(Game(tablero, u1)[0]):
                    tablero
                u2 = random.randint(0,5)
                Game(tablero)
    


In [38]:
#Reglas
### Recibe tablero y next move (intento de next move)
### Retorna tablero y si se puede hacer el next move
def ValideMove(tablero, player, next_move):
    # evaluar si es posible realizar el next_move
    if(tablero.board[player][next_move] != 0):
        return True
    else:
        return False
    
    
def Reglas(tablero, player, next_move):
    if(player == 0):
        adversario = 1
    else:
        adversario = 0
    
    if(tablero.fila_Vacia(adversario) and not tablero.fila_Vacia(player)):
        # la fila del adversario está vacía
        # se traslandan los tokens de la fila del player a su mancala
        tokens_1 = len(tablero.board[player]) #solo devuelve la cantidad de casillas
        tablero.mancala[player] = tokens_1
        # se eliminan los tokens de la fila 1
        for i in range(6):
            tablero.board[player][i] = 0
    
    [dire, last] = tablero.makePlay(player, next_move)
    if(last >= 0 and last <=5):
        if(tablero.board[player][last] == 1 and dire == adversario): #quiere decir que antes estaba vacío y está del lado del jugador (dire lógica inversa)
            #ROBAR FICHAS
            print("holis")
            tokens_a = tablero.board[adversario][last]
            if(tokens_a > 0):
                tablero.board[adversario][last] = 0          #se roban fichas del adversario
                tablero.board[player][last] = 0              #ficha del jugador al mancala
                tablero.mancala[player] += tokens_a + 1
        
    if(last == -1 or last == 6): # último token en mancala del jugador
        next_player = player
        print("MANCALA!!!")
    else:
        next_player = adversario
        print("MANCALA!!!")
        
        
            
    return next_player # necesita retornar tablero? por ahora no

In [12]:
IA(tablero, possible_moves):
    n = 100
    tablero0 = tablero
    cantMoves = len(possible_moves)
    for i in possible_moves:
        for j in range(n):
            tablero0.makePlay(0,i)
            while(not tablero0.over):
                tablero.makePlay(1,random.randint(0,5))
                
        
    

SyntaxError: invalid syntax (<ipython-input-12-99a286f1788e>, line 1)

In [39]:
def Game():
    # determinar primer jugador
    player = random.randint(0,1)
    if(player == 0):
        print("La inteligencia artificial iniciará este juego")
    else:
        print("Tienes el primer turno")
        
    tablero = Tablero()    
    while(not tablero.over):
        tablero.showBoard()
        if(player == 0):
            possible_moves = []
            for i in range(6):
                if(ValideMove(tablero, player, i)):
                    possible_moves.append(i)
            print("Movimientos posibles", possible_moves)
            next_move = int(input("Juego computadora "))
            #next_move = IA(tablero, possible_moves)
        else:
            valido = False
            while(not valido):
                x = int(input("¿Qué casilla deseas mover? "))
                if(ValideMove(tablero,player,x)):
                    next_move = x
                    valido = True
                else:
                    print("No hay fichas en esa casilla, elige otra.")
                    valido = False
        tablero.gameOver()
                    
        #tablero.makePlay(player,next_move)
        player = Reglas(tablero, player, next_move)
    
    print("GAME OVER")
            
            

In [6]:
#test block
game = Tablero()
game.showBoard()

game.makePlay(0,5)
game.showBoard()
game.makePlay(0,3)
game.showBoard()
game.makePlay(0,2)
game.showBoard()
game.makePlay(0,1)
game.showBoard()
game.makePlay(0,0)
game.showBoard()
game.makePlay(0,1)
game.showBoard()


NameError: name 'clear_output' is not defined

In [40]:
Game()

Tienes el primer turno
_______________________ *** _______________________
Jugador 1:  0 			Jugador 2:  0 


	4	4	4	4	4	4	

	4	4	4	4	4	4	

¿Qué casilla deseas mover? 0
last position:  4
MANCALA!!!
_______________________ *** _______________________
Jugador 1:  0 			Jugador 2:  0 


	4	4	4	4	4	4	

	0	5	5	5	5	4	

Movimientos posibles [0, 1, 2, 3, 4, 5]
Juego computadora 3
last position:  -1
MANCALA!!!
_______________________ *** _______________________
Jugador 1:  1 			Jugador 2:  0 


	5	5	5	0	4	4	

	0	5	5	5	5	4	

Movimientos posibles [0, 1, 2, 4, 5]
Juego computadora 0
last position:  3
MANCALA!!!
_______________________ *** _______________________
Jugador 1:  2 			Jugador 2:  0 


	0	5	5	0	4	4	

	1	6	6	6	5	4	



KeyboardInterrupt: 